# Trend detection in online news

1. Schritt: News eines Zeitraums (Tag, Woche, Monat, Jahr) einlesen
2. Schritt: Sprachbestimmung jedes News-Beitrags, Tokenisierung, Lemmatisierung (Stammwortbildung), nur Substantive und Eigennamen verwenden

In [15]:
# 1. Schritt
import os

available_periods = ['day','week','month','year']
period = 'month'
f = open(os.getcwd() + '/data/latest-news-of-' + period +'.txt','r')
news = f.readlines()
f.close()
print("{} eingelesene News".format(len(news)))

# 2. Schritt
import spacy
from langdetect import detect

# Vorerst nur deutsch und english unterstützen
nlp_de = spacy.load('de')
nlp_en = spacy.load('en')

raw_documents = []
for i in range(len(news)):
    # Simple language detection
    lang = detect(news[i])
    doc = ''
    if lang == 'de':
        doc = nlp_de(news[i])
    else:
        doc = nlp_en(news[i])
    # print("{}: {}".format(lang, news[i]))
    
    nouns = set()
    for token in doc:
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
            nouns.add(token.lemma_)
    raw_documents.append(" ".join(nouns))

print("Datenaufbereitung abgeschlossen!")    

2885 eingelesene News
Datenaufbereitung abgeschlossen!


3. Schritt: Bag-Of-Words-Repräsentation erstellen
4. Schritt: Latent Dirchilet Allocation (LDA) zum Clustering anwenden

In [18]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
warnings.filterwarnings(action='ignore', category=FutureWarning, module='pyLDAvis')

from gensim import corpora, models
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

# Dictionary erstellen
dictionary = corpora.Dictionary([" ".join(raw_documents).split()]) 

print('{} verschiedene Wörter im Corpus'.format(len(dictionary)))

bow_corpus = [dictionary.doc2bow(text.split()) for text in raw_documents]

total_topics = 7
    
lda_model_bow = models.LdaModel(corpus=bow_corpus,alpha='auto', id2word=dictionary, num_topics=total_topics,
                                passes=5, random_state=47)

print("News-Clustering abgeschlossen!")

data = pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, dictionary)

pyLDAvis.display(data)

11661 verschiedene Wörter im Corpus
News-Clustering abgeschlossen!
